In [33]:
import arcpy
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

In [34]:
gdb_path = r"C:\Users\rtvpd\OneDrive\Documentos\ArcGIS\Projects\Walkability_Seattle\Walkability_Seattle.gdb"

# Set the workspace to the geodatabase
arcpy.env.workspace = gdb_path

# List all feature classes in the geodatabase
feature_classes = arcpy.ListFeatureClasses()
print("Feature Classes in the Geodatabase:")
for fc in feature_classes:
    print(fc)

Feature Classes in the Geodatabase:
Polygons_2
WorldImagery_DetectObjectsUs
Polygons_2_1
WorldImagery_DetectObjectsUs1
Polygons_2_2
WorldImagery_DetectObjectsUs2
limits_JSONToFeatures
citylimits_JSONToFeatures
citylimits_JSONToFeatur_Clip
Park_Boundarie_JSONToFeature
Parks_Intersect
Parks_Boundary_out_Intersect
Polygons_2_3
fishnet_park_Intersect
fishnet_park_Inter_Intersect
fishnet_park_Inter_Intersect1
Walkability
Polygons_2_4
Polygons_2_5
fishnet_inter
citylimits
fishnet_Intersect
fishnet_Intersect1
Parks_Boundary_out_Intersect1
Park_Boundarie_JSONToFeature1
Parks_SPR_Merge
Parks_Merge_Intersect
Parks_Merge_Intersect1
fishnet_Intersect2
Polygons_2_6
Polygons_2_7
Parks_Merge_Intersect2
fishnet_sidewalk_intersect
fishnet_sidewalks_Intersect
Seattle_Streets
Seattle_Streets_poly
trails_Intersect
fishnet_ExportFeatures
fishnet_dataframe
Streets_Intersect
Bike_facilities
shp
Bike_facilities_BKF_BL
Bike_facilities_BKF_CLMB
Bike_facilities_BKF_OFFST
Bike_facilities_BKF_SHW
Bike_Greenways_In

In [35]:
gdb_path = r"C:\Users\rtvpd\OneDrive\Documentos\ArcGIS\Projects\Walkability_Seattle\Walkability_Seattle.gdb"
exported_fishnet_layer = r"C:\Users\rtvpd\OneDrive\Documentos\ArcGIS\Projects\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet"

# Set the workspace to the geodatabase
arcpy.env.workspace = gdb_path

# List all fields in the exported fishnet layer
fields = arcpy.ListFields(exported_fishnet_layer)

# Print field names and types
print("Field Names and Types:")
for field in fields:
    print(f"Name: {field.name}, Type: {field.type}, Alias: {field.aliasName}")

Field Names and Types:
Name: OBJECTID, Type: OID, Alias: OID
Name: Shape, Type: Geometry, Alias: Shape
Name: IndexID, Type: Integer, Alias: IndexID
Name: Shape_Length, Type: Double, Alias: Shape_Length
Name: Shape_Area, Type: Double, Alias: Shape_Area
Name: total_area, Type: Double, Alias: total_area
Name: Industrial_SUM_Industrial_effective_area, Type: Double, Alias: Industrial_SUM_Industrial_effective_area
Name: ParkingLots_SUM_ParkingLots_effective_area, Type: Double, Alias: ParkingLots_SUM_ParkingLots_effective_area
Name: GolfCourse_SUM_GolfCourse_effective_area, Type: Double, Alias: GolfCourse_SUM_GolfCourse_effective_area
Name: Cemeteries_SUM_Cemeteries_effective_area, Type: Double, Alias: Cemeteries_SUM_Cemeteries_effective_area
Name: Hospitals_SUM_Hospitals_effective_area, Type: Double, Alias: Hospitals_SUM_Hospitals_effective_area
Name: Slope_MEAN, Type: Double, Alias: Slope_MEAN
Name: Bike_greenways_SUM_Bike_greenways_area, Type: Double, Alias: Bike_greenways_SUM_Bike_greenwa

In [36]:
field_list = []
slope_field = "Slope_MEAN"

In [37]:
for field in arcpy.ListFields(exported_fishnet_layer):
    if "area" in field.name.lower() and field.name.lower() != "shape_area":
        field_list.append(field.name)

# Print the populated field_list
print("fields in walkscore_fishnet:")
for field in field_list:
    print(field)
field_list.append(slope_field)  # Add slope field to the list

fields in walkscore_fishnet:
total_area
Industrial_SUM_Industrial_effective_area
ParkingLots_SUM_ParkingLots_effective_area
GolfCourse_SUM_GolfCourse_effective_area
Cemeteries_SUM_Cemeteries_effective_area
Hospitals_SUM_Hospitals_effective_area
Bike_greenways_SUM_Bike_greenways_area
Bike_protected_SUM_Bike_protected_area
Bike_buffer_SUM_Bike_buffer_area
Healthy_Streets_SUM_Healthy_Streets_area
Parks_SUM_Parks_area
Universities_SUM_Universities_area
Sidewalks_SUM_Sidewalks_area
Plaza_SUM_Plaza_area
trails_SUM_trails_area
MultiUseTrails_SUM_MultiUseTrails_area
Streets_SUM_Streets_effective_area


In [38]:
def normalize_field(layer, field_name, norm_field_name, inverse=False, fill_null_value=None):
    try:
        # Calculate min and max values for the specified field
        with arcpy.da.SearchCursor(layer, [field_name]) as cursor:
            values = [row[0] for row in cursor if row[0] is not None]
            if not values:
                print(f"No values found for {field_name}. Skipping normalization.")
                return
            min_value = min(values)
            max_value = max(values)
        
        # Add new field for normalized values if it does not already exist
        if not any(f.name == norm_field_name for f in arcpy.ListFields(layer)):
            arcpy.management.AddField(layer, norm_field_name, "DOUBLE")
        
        # Normalize the data and store in the new field
        with arcpy.da.UpdateCursor(layer, [field_name, norm_field_name]) as cursor:
            for row in cursor:
                if row[0] is not None:
                    normalized_value = (row[0] - min_value) / (max_value - min_value)
                    if inverse:
                        normalized_value = 1 - normalized_value
                    row[1] = normalized_value
                else:
                    row[1] = fill_null_value if fill_null_value is not None else None
                cursor.updateRow(row)
        print(f"Normalization complete for {field_name}.")
    except Exception as e:
        print(f"Error processing field {field_name}: {e}")

In [39]:
for field in field_list:
    norm_field = "NORM_" + field
    if field == slope_field:
        normalize_field(walkscore_fishnet, 'Slope_Mean', 'NORM_Slope_Mean', inverse=True, fill_null_value=1)
    else:
        normalize_field(walkscore_fishnet, field, norm_field)

Normalization complete for total_area.
Normalization complete for Industrial_SUM_Industrial_effective_area.
Normalization complete for ParkingLots_SUM_ParkingLots_effective_area.
Normalization complete for GolfCourse_SUM_GolfCourse_effective_area.
Normalization complete for Cemeteries_SUM_Cemeteries_effective_area.
Normalization complete for Hospitals_SUM_Hospitals_effective_area.
Normalization complete for Bike_greenways_SUM_Bike_greenways_area.
Normalization complete for Bike_protected_SUM_Bike_protected_area.
Normalization complete for Bike_buffer_SUM_Bike_buffer_area.
Normalization complete for Healthy_Streets_SUM_Healthy_Streets_area.
Normalization complete for Parks_SUM_Parks_area.
Normalization complete for Universities_SUM_Universities_area.
Normalization complete for Sidewalks_SUM_Sidewalks_area.
Normalization complete for Plaza_SUM_Plaza_area.
Normalization complete for trails_SUM_trails_area.
Normalization complete for MultiUseTrails_SUM_MultiUseTrails_area.
Normalization co

# Calculating Walkscore

#### Setup

In [40]:
sidewalk_score_field = 'sidewalk_score'
park_score_field = 'park_score'
trail_score_field = 'trail_score'
street_score_field = 'street_score'
bike_score_field = 'bike_score'
walkscore_field = 'walk_score'
unadjusted_walkscore_field = 'unadjusted_walkscore'
slope_field = "Slope_MEAN"

In [41]:
positive_weights = {
    sidewalk_score_field: 0.55,
    park_score_field: 0.1,
    trail_score_field: 0.3,
    bike_score_field: 0.05
}

In [42]:
negative_weights = {
    "effective_area_scaler": 1,
}

In [43]:
assert abs(sum(positive_weights.values()) - 1.0) < 1e-6, "The positive weights must sum up to 1.0"

In [44]:
for score_field in [walkscore_field,sidewalk_score_field,park_score_field,trail_score_field,street_score_field,bike_score_field,unadjusted_walkscore_field]:
    if not any(f.name == score_field for f in arcpy.ListFields(walkscore_fishnet)):
        arcpy.management.AddField(walkscore_fishnet, score_field, "DOUBLE")

### Slope Scaler

As part of the data, we have a slope element that describes how steep the slope is within each fishnet grid. Instead of handling this element like the others, I'd like to implement a custom function that will allow me to more readily customize the output of the slope scaler.

In [45]:
with arcpy.da.UpdateCursor(walkscore_fishnet, [slope_field]) as cursor:
    for row in cursor:
        if row[0] is None:
            row[0] = 0
        cursor.updateRow(row)

print("Populated null values in Slope_Mean field with 0.")

Populated null values in Slope_Mean field with 0.


In [46]:
# Field containing the slope values
slope_field = "Slope_Mean"

# List to store slope values
slope_values = []

# Extract slope values
with arcpy.da.SearchCursor(exported_fishnet_layer, [slope_field]) as cursor:
    for row in cursor:
        if row[0] is not None:
            slope_values.append(row[0])

print(f"Extracted {len(slope_values)} slope values.")

Extracted 31701 slope values.


In [47]:
slope_series = pd.Series(slope_values)
slope_stats = slope_series.describe()
print("Descriptive Statistics for Slope Values:")
print(slope_stats)

Descriptive Statistics for Slope Values:
count    31701.000000
mean         5.009523
std          4.656576
min          0.000000
25%          1.833370
50%          3.612894
75%          6.744153
max         44.830341
dtype: float64


In [48]:
def get_slope_scaler(slope_value):
    if slope_value is None or slope_value < 0:
        return 1.0
    elif slope_value < 2:
        return 1.0
    elif 2 <= slope_value < 3:
        return 0.95
    elif 3 <= slope_value < 4:
        return 0.9
    elif 4 <= slope_value < 5:
        return 0.7
    elif 5 <= slope_value < 7:
        return 0.5
    elif 7 <= slope_value < 10:
        return 0.3
    elif 10 <= slope_value < 15:
        return 0.1
    elif 15 <= slope_value < 20:
        return 0.05
    elif 20 <= slope_value < 25:
        return 0.02
    else:
        return 0.0

### Streets, ParkingLots, and Industrial Scaler Values

In the previous sheet, I created "effective areas" for each of Streets, Industrial, and Parkinglots. These effective area represent the area of surface streets in each fishnet grid multiplied by the speed limit on each street. Since Industrial and Parklots don't have explicit speed limits, I've set a universal speed value of 15 and used this to calculate the effective area.

Below, I'll create a scaler value based on the sum of all three effective area fields.

In [151]:
streets_field = "Streets_SUM_Streets_effective_area"
industrial_field = "Industrial_SUM_Industrial_effective_area"
parkinglots_field = "ParkingLots_SUM_Parkinglots_effective_area"

In [152]:
combined_values = []

In [153]:
with arcpy.da.SearchCursor(exported_fishnet_layer, [streets_field, industrial_field, parkinglots_field]) as cursor:
    for row in cursor:
        combined_value = (row[0] if row[0] is not None else 0) + \
                         (row[1] if row[1] is not None else 0) + \
                         (row[2] if row[2] is not None else 0)
        combined_values.append(combined_value)

print(f"Extracted {len(combined_values)} combined values.")

Extracted 31701 combined values.


In [154]:
combined_series = pd.Series(combined_values)
combined_stats = combined_series.describe()

In [155]:
mean_combined = combined_stats['mean']
std_combined = combined_stats['std']

def normalize_combined_value(combined_value):
    return (combined_value - mean_combined) / std_combined

In [156]:
normalized_combined_values = [normalize_combined_value(value) for value in combined_values]
normalized_combined_series = pd.Series(normalized_combined_values)
normalized_combined_stats = normalized_combined_series.describe()

In [157]:
print("Descriptive Statistics for Combined Values:")
print(normalized_combined_stats)

Descriptive Statistics for Combined Values:
count    3.170100e+04
mean     9.503489e-17
std      1.000000e+00
min     -7.170337e-01
25%     -4.809313e-01
50%     -2.922593e-01
75%      8.381303e-02
max      1.989856e+01
dtype: float64


In [158]:
combined_series.isna().sum()

0

In [159]:
def test_get_effective_area_scaler(normalized_combined_value, decay_rate):
    return 1 / (1 + math.exp(decay_rate * normalized_combined_value))

In [160]:
min_value = normalized_combined_stats['min']
max_value = normalized_combined_stats['max']
percentile_25 = normalized_combined_stats['25%']
percentile_75 = normalized_combined_stats['75%']

# Adjust linspace to focus more on the range between 25th and 75th percentile
combined_values_for_plot = np.linspace(min_value, max_value, 1000)

In [161]:
# Decay rates to visualize
decay_rates = [5,10,15,20,30,35]

# Plot the scaler values for different decay rates
plt.figure(figsize=(10, 6))

for decay_rate in decay_rates:
    scaler_values = [test_get_effective_area_scaler(value, decay_rate) for value in combined_values_for_plot]
    plt.plot(combined_values_for_plot, scaler_values, label=f"Decay Rate = {decay_rate}")

plt.xlabel('Normalized Combined Value')
plt.ylabel('Scaler Value')
plt.title('Effect of Decay Rate on Effective Area Scaler')
plt.legend()
plt.grid(True)
plt.xlim(percentile_25, percentile_75)  # Focus on the range between 25th and 75th percentile
plt.show()

#### Defining Final Effective Area Scaler Function

In [163]:
def get_effective_area_scaler(combined_value):
    if combined_value is None:
        return 1.0
    else:
        normalized_value = normalize_combined_value(combined_value)
        decay_rate = 15  # Adjust as needed for a more gradual effect
        return 1 / (1 + math.exp(decay_rate * normalized_value))

#### Business Density Scaler

In [164]:
business_density = 'business_density'

In [165]:
# List to store slope values
business_values = []

# Extract slope values
with arcpy.da.SearchCursor(exported_fishnet_layer, [business_density]) as cursor:
    for row in cursor:
        if row[0] is not None:
            business_values.append(row[0])

print(f"Extracted {len(business_values)} business density values.")

Extracted 31701 business density values.


In [166]:
business_series = pd.Series(business_values)
business_density_stats = business_series.describe()
print("Descriptive Statistics for Business Values:")
print(business_density_stats)

Descriptive Statistics for Business Values:
count    31701.000000
mean         0.000022
std          0.000126
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          0.003659
dtype: float64


From the distribution above, we can see that the majority of values are 0. This indicates that most fishnet grids don't have a business in them.

Because of this distribution, I'd like to reward fishnet grids that contain businesses and set the minimum scaler to 1. Below I'll adjust the business density scaler to reflect this, using 1 as the minimum and 2 as the maximum.

In [167]:
def normalize_business_density(business_density_value):
    mean_business_density = business_density_stats['mean']
    std_business_density = business_density_stats['std']
    return (business_density_value - mean_business_density) / std_business_density

In [168]:
def get_business_density_scaler(business_density_value):
    if business_density_value is None or business_density_value <= 0:
        return 1.0
    elif business_density_value >= 0.0015:
        return 2.0
    else:
        # Linearly interpolate between 1 and 2 for values between 0 and 0.00428
        return 1.0 + (business_density_value / 0.0015)

#### Public Amenity Density Scaler

In [169]:
public_amenity_density = 'public_density'

In [170]:
# List to store slope values
public_amenity_values = []

# Extract slope values
with arcpy.da.SearchCursor(exported_fishnet_layer, [public_amenity_density]) as cursor:
    for row in cursor:
        if row[0] is not None:
            public_amenity_values.append(row[0])

print(f"Extracted {len(public_amenity_values)} Public Amenity Density values.")

Extracted 31701 Public Amenity Density values.


In [171]:
public_amenity_series = pd.Series(public_amenity_values)
public_amenity_density_stats = public_amenity_series.describe()
print("Descriptive Statistics for Public Amenity Density Values:")
print(public_amenity_density_stats)

Descriptive Statistics for Public Amenity Density Values:
count    31701.000000
mean         0.000034
std          0.000224
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          0.015842
dtype: float64


In [172]:
def normalize_public_amenity_density(public_amenity_density_value):
    mean_public_amenity_density = public_amenity_density_stats['mean']
    std_public_amenity_density = public_amenity_density_stats['std']
    return (public_amenity_density_value - mean_public_amenity_density) / std_public_amenity_density

In [173]:
def get_public_amenity_density_scaler(public_amenity_density_value):
    if public_amenity_density_value is None or public_amenity_density_value <= 0:
        return 1.0
    elif public_amenity_density_value >= 0.00428:
        return 1.5
    else:
        # Linearly interpolate between 1 and 1.5 for values between 0 and 0.00428
        return 1.0 + (0.5 * public_amenity_density_value / 0.00428)

### Creating Scaler Fields for Troubleshooting

In [174]:
scaler_fields = {
    'slope_scaler': 'slope_scaler',
    'effective_area_scaler': 'effective_area_scaler',
    'business_density_scaler': 'business_density_scaler',
    'public_amenity_density_scaler': 'public_amenity_density_scaler'
}

In [175]:
industrial_field = "NORM_Industrial_SUM_Industrial_effective_area"
golfcourses_field = "NORM_GolfCourse_SUM_GolfCourse_effective_area"
hospitals_field = "NORM_Hospitals_SUM_Hospitals_effective_area"
cemeteries_field = "NORM_Cemeteries_SUM_Cemeteries_effective_area"

parkinglots_field = "NORM_ParkingLots_SUM_Parkinglots_effective_area"
streets_field = "NORM_Streets_SUM_Streets_effective_area"
slope_field = "Slope_MEAN"
walkscore_field = "walk_score"
unadjusted_walkscore_field = "unadjusted_walkscore"
public_amenity_density_field = 'public_density'
business_density_field = 'business_density'

In [176]:
score_fields = [sidewalk_score_field, park_score_field, trail_score_field, bike_score_field, walkscore_field, unadjusted_walkscore_field]
amenities_fields = [business_density, public_amenity_density]

In [177]:
fields = [streets_field, industrial_field, golfcourses_field, hospitals_field, cemeteries_field, parkinglots_field, slope_field, walkscore_field] + list(scaler_fields.values())+ list(score_fields)+list(amenities_fields)

In [178]:
fields

['NORM_Streets_SUM_Streets_effective_area', 'NORM_Industrial_SUM_Industrial_effective_area', 'NORM_GolfCourse_SUM_GolfCourse_effective_area', 'NORM_Hospitals_SUM_Hospitals_effective_area', 'NORM_Cemeteries_SUM_Cemeteries_effective_area', 'NORM_ParkingLots_SUM_Parkinglots_effective_area', 'Slope_MEAN', 'walk_score', 'slope_scaler', 'effective_area_scaler', 'business_density_scaler', 'public_amenity_density_scaler', 'sidewalk_score', 'park_score', 'trail_score', 'bike_score', 'walk_score', 'unadjusted_walkscore', 'business_density', 'public_density']

In [179]:
# Add new scaler fields if they don't exist
for field_name in scaler_fields.values():
    if not any(f.name == field_name for f in arcpy.ListFields(walkscore_fishnet)):
        arcpy.management.AddField(walkscore_fishnet, field_name, "DOUBLE")

In [180]:
# Update scalers in walkscore_fishnet
with arcpy.da.UpdateCursor(walkscore_fishnet, fields) as cursor:
    for row in cursor:
        slope_value = row[fields.index(slope_field)]
        streets_value = row[fields.index(streets_field)]
        industrial_value = row[fields.index(industrial_field)]
        parkinglots_value = row[fields.index(parkinglots_field)]
        golfcourses_value = row[fields.index(golfcourses_field)]
        hospitals_value = row[fields.index(hospitals_field)]
        cemeteries_value = row[fields.index(cemeteries_field)]
        business_density_value = row[fields.index(business_density_field)]
        public_amenity_density_value = row[fields.index(public_amenity_density_field)]

        combined_value = (streets_value if streets_value is not None else 0) + \
                         (industrial_value if industrial_value is not None else 0) + \
                         (parkinglots_value if parkinglots_value is not None else 0) + \
                         (golfcourses_value if golfcourses_value is not None else 0) + \
                         (hospitals_value if hospitals_value is not None else 0) + \
                         (cemeteries_value if cemeteries_value is not None else 0)
        
        slope_scaler = get_slope_scaler(slope_value)
        effective_area_scaler = get_effective_area_scaler(combined_value)
        business_density_scaler = get_business_density_scaler(business_density_value)
        public_amenity_density_scaler = get_public_amenity_density_scaler(public_amenity_density_value)

        row[fields.index(scaler_fields['slope_scaler'])] = slope_scaler
        row[fields.index(scaler_fields['effective_area_scaler'])] = effective_area_scaler
        row[fields.index(scaler_fields['business_density_scaler'])] = business_density_scaler
        row[fields.index(scaler_fields['public_amenity_density_scaler'])] = public_amenity_density_scaler
        
        cursor.updateRow(row)

print("Scaler values updated and stored in walkscore_fishnet.")

Scaler values updated and stored in walkscore_fishnet.


### Score Component Definitions

In [181]:
sidewalk_score_components = ['NORM_Sidewalks_SUM_Sidewalks_area',"NORM_Plaza_SUM_Plaza_area"]
park_score_components = ['NORM_Parks_SUM_parks_area', 'NORM_Universities_SUM_Universities_area']
trail_score_components = ['NORM_trails_SUM_trails_area','NORM_MultiUseTrails_SUM_MultiUseTrails_area']
bike_score_components = ['NORM_Bike_greenways_SUM_Bike_greenways_area',
                         'NORM_Bike_protected_SUM_Bike_protected_area',
                         'NORM_Bike_buffer_SUM_Bike_buffer_area',
                         'NORM_Healthy_Streets_SUM_Healthy_Streets_area']
street_score_components = ['NORM_Streets_SUM_Streets_effective_area']
slope_component = [slope_field]

In [182]:
walkscore_fields = (sidewalk_score_components + park_score_components + trail_score_components +
          bike_score_components + street_score_components + slope_component +
          score_fields + list(scaler_fields.values()))

In [183]:
for field_name in [walkscore_field, unadjusted_walkscore_field]:
    if not any(f.name == field_name for f in arcpy.ListFields(walkscore_fishnet)):
        arcpy.management.AddField(walkscore_fishnet, field_name, "DOUBLE")
        walkscore_fields.append(field_name)  # Add the new field to the list

In [184]:
# Calculate walkscore using the effective area scaler
with arcpy.da.UpdateCursor(walkscore_fishnet, walkscore_fields) as cursor:
    for row in cursor:
        # Calculate individual scores
        sidewalk_score = sum(row[walkscore_fields.index(component)] for component in sidewalk_score_components if row[walkscore_fields.index(component)] is not None)
        park_score = sum(row[walkscore_fields.index(component)] for component in park_score_components if row[walkscore_fields.index(component)] is not None)
        trail_score = sum(row[walkscore_fields.index(component)] for component in trail_score_components if row[walkscore_fields.index(component)] is not None)
        bike_score = sum(row[walkscore_fields.index(component)] for component in bike_score_components if row[walkscore_fields.index(component)] is not None)

        row[walkscore_fields.index(sidewalk_score_field)] = sidewalk_score
        row[walkscore_fields.index(park_score_field)] = park_score
        row[walkscore_fields.index(trail_score_field)] = trail_score
        row[walkscore_fields.index(bike_score_field)] = bike_score

        slope_scaler = row[walkscore_fields.index(scaler_fields['slope_scaler'])]
        effective_area_scaler = row[walkscore_fields.index(scaler_fields['effective_area_scaler'])]
        business_density_scaler = row[walkscore_fields.index(scaler_fields['business_density_scaler'])]
        public_amenity_density_scaler = row[walkscore_fields.index(scaler_fields['public_amenity_density_scaler'])]

        # Calculate positive component of the walkscore (unadjusted walkscore)
        walkscore_positive = sum(row[walkscore_fields.index(field)] * positive_weights[field] for field in score_fields if row[walkscore_fields.index(field)] is not None)

        # Debugging statements
#         print(f"Walkscore Positive: {walkscore_positive}")
#         print(f"Effective Area Scaler: {effective_area_scaler}")
#         print(f"Slope Scaler: {slope_scaler}")
#         print(f"Business Density Scaler: {business_density_scaler}")
#         print(f"Public Amenity Density Scaler: {public_amenity_density_scaler}")

        # Assign the unadjusted walkscore to the unadjusted_walkscore field
        row[walkscore_fields.index(unadjusted_walkscore_field)] = walkscore_positive

        # Calculate final walkscore with scalers applied
        walkscore = walkscore_positive * effective_area_scaler * slope_scaler * business_density_scaler * public_amenity_density_scaler

        # Debugging statement
#         print(f"Calculated Walkscore: {walkscore}")

        # Assign the calculated walkscore to the walkscore field explicitly
        row[walkscore_fields.index(walkscore_field)] = walkscore
        
        cursor.updateRow(row)

print("Aggregated and scaled walkscore calculation complete.")

Walkscore Positive: 0.021746899004593135
Effective Area Scaler: 0.9999786727813997
Slope Scaler: 0.7
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.015222504642606929
Walkscore Positive: 0.05669963378977255
Effective Area Scaler: 0.9999786727508757
Slope Scaler: 0.95
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.053863503315429585
Walkscore Positive: 0.05458276528118631
Effective Area Scaler: 0.99997867267592
Slope Scaler: 0.95
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.05185252111801888
Walkscore Positive: 0.17711390550888786
Effective Area Scaler: 0.9999786725146519
Slope Scaler: 1.0
Business Density Scaler: 1.0937541254510035
Public Amenity Density Scaler: 1.0164288771234236
Calculated Walkscore: 0.19689745212047297
Walkscore Positive: 0.035171848538220446
Effective Area Scaler: 0.9999786727721827
Slope Scaler: 0.5
Business Density Scaler: 1.0
Public Amenit

Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.0
Walkscore Positive: 0.005411858069303301
Effective Area Scaler: 0.9999786727782028
Slope Scaler: 0.3
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.0016235227948217768
Walkscore Positive: 0.005652213784472933
Effective Area Scaler: 0.9999786727886163
Slope Scaler: 0.5
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.002826046619257383
Walkscore Positive: 0.026402756427276946
Effective Area Scaler: 0.9999786727734771
Slope Scaler: 1.0
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.02640219332970979
Walkscore Positive: 0.0
Effective Area Scaler: 0.9999786726436378
Slope Scaler: 0.95
Business Density Scaler: 1.2064569324391352
Public Amenity Density Scaler: 1.036178200777886
Calculated Walkscore: 0.0
Walkscore Positive: 0.16155373985806332
Effective Area Scaler: 0.999978672790038
S

Effective Area Scaler: 0.9999786727697747
Slope Scaler: 0.9
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.0341271665970799
Walkscore Positive: 0.029043892074059285
Effective Area Scaler: 0.9999786727560809
Slope Scaler: 0.3
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.008712981794366598
Walkscore Positive: 0.03818487707811563
Effective Area Scaler: 0.9999786727711248
Slope Scaler: 0.9
Business Density Scaler: 1.3945605963772383
Public Amenity Density Scaler: 1.0691402914212451
Calculated Walkscore: 0.051238538124084715
Walkscore Positive: 0.0
Effective Area Scaler: 0.9999786727859623
Slope Scaler: 1.0
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.0
Walkscore Positive: 0.029560210557672384
Effective Area Scaler: 0.9999786727705171
Slope Scaler: 0.7
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.02069170608419478
Walkscore

Walkscore Positive: 0.04538685319345982
Effective Area Scaler: 0.9999786727630612
Slope Scaler: 0.9
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.040847296695559074
Walkscore Positive: 0.020885509139852126
Effective Area Scaler: 0.9999786727806644
Slope Scaler: 0.5
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.010442531855008883
Walkscore Positive: 0.050589031588740625
Effective Area Scaler: 0.9999786726216369
Slope Scaler: 0.7
Business Density Scaler: 1.2815538427039437
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.04538182958576214
Walkscore Positive: 0.04614427156047589
Effective Area Scaler: 0.999978672770484
Slope Scaler: 0.9
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.041528958687904925
Walkscore Positive: 0.14130613377973567
Effective Area Scaler: 0.999978672790038
Slope Scaler: 0.5
Business Density Scaler: 1.0
Public Amenity Density Scaler

Business Density Scaler: 1.3756969667155146
Public Amenity Density Scaler: 1.148128184890755
Calculated Walkscore: 0.04624951621857411
Walkscore Positive: 0.026056436772025973
Effective Area Scaler: 0.9999786727668949
Slope Scaler: 0.9
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.023450292954292542
Walkscore Positive: 0.04638937383362084
Effective Area Scaler: 0.9999786727541852
Slope Scaler: 0.9
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.0417495460284377
Walkscore Positive: 0.03065473101354548
Effective Area Scaler: 0.9999786727636873
Slope Scaler: 1.0
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.049469998593882
Calculated Walkscore: 0.03217053439045904
Walkscore Positive: 0.08068213321868038
Effective Area Scaler: 0.9999786727756821
Slope Scaler: 1.0
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.08068041249272678
Walkscore Positive: 0.0240

...

Calculated Walkscore: 0.0016474051436433548
Walkscore Positive: 0.0
Effective Area Scaler: 0.9999786727716229
Slope Scaler: 1.0
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.0
Walkscore Positive: 0.00031002009050587736
Effective Area Scaler: 0.9999786724522751
Slope Scaler: 1.0
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.0003100134785376014
Walkscore Positive: 0.025997588312688325
Effective Area Scaler: 0.9999786727450619
Slope Scaler: 1.0
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.025997033855494605
Walkscore Positive: 0.059642322924320645
Effective Area Scaler: 0.9999786727802605
Slope Scaler: 0.9
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.053676945827454484
Walkscore Positive: 0.03757710370160096
Effective Area Scaler: 0.9999786727802781
Slope Scaler: 0.9
Business Density Scaler: 1.0
Public Amenity Density Sca

Effective Area Scaler: 0.999978672790038
Slope Scaler: 0.05
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.0037245453767035712
Walkscore Positive: 0.0033295117201362495
Effective Area Scaler: 0.999978672790038
Slope Scaler: 0.3
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.000998832213282217
Walkscore Positive: 0.0
Effective Area Scaler: 0.9999786727822138
Slope Scaler: 0.1
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.0
Walkscore Positive: 0.035579207073788574
Effective Area Scaler: 0.9999786727698275
Slope Scaler: 0.95
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.03379952585445746
Walkscore Positive: 0.0
Effective Area Scaler: 0.999978672689189
Slope Scaler: 1.0
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.0
Walkscore Positive: 0.0235113602365483
Effective Area Scaler: 0.99997

Slope Scaler: 0.3
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.0037659000454375166
Walkscore Positive: 0.2624462766227989
Effective Area Scaler: 0.9999786724641562
Slope Scaler: 0.7
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0164520560882286
Calculated Walkscore: 0.18673085764616223
Walkscore Positive: 0.020927917728290814
Effective Area Scaler: 0.999978672766556
Slope Scaler: 0.95
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.01988109782401873
Walkscore Positive: 0.0
Effective Area Scaler: 0.9999786724472756
Slope Scaler: 1.0
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.0
Walkscore Positive: 0.025120927815789586
Effective Area Scaler: 0.9999786727775106
Slope Scaler: 0.95
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.02386437245336427
Walkscore Positive: 0.025132829305493043
Effective Area Scaler: 0

Walkscore Positive: 0.054400789533447215
Effective Area Scaler: 0.9999786727700243
Slope Scaler: 0.9
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.048959666383768176
Walkscore Positive: 0.0
Effective Area Scaler: 0.999978672458534
Slope Scaler: 1.0
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.0
Walkscore Positive: 0.028881174735350266
Effective Area Scaler: 0.9999786727589258
Slope Scaler: 0.95
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.02743653084059547
Walkscore Positive: 0.0
Effective Area Scaler: 0.999978672439469
Slope Scaler: 1.0
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.0
Walkscore Positive: 0.009354410008742025
Effective Area Scaler: 0.9999786727886858
Slope Scaler: 0.1
Business Density Scaler: 1.0
Public Amenity Density Scaler: 1.0
Calculated Walkscore: 0.0009354210505263049
Walkscore Positive: 0.0
Effec

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [185]:
min_walkscore, max_walkscore = None, None
with arcpy.da.SearchCursor(walkscore_fishnet, [walkscore_field]) as cursor:
    for row in cursor:
        if min_walkscore is None or row[0] < min_walkscore:
            min_walkscore = row[0]
        if max_walkscore is None or row[0] > max_walkscore:
            max_walkscore = row[0]

# Normalize the scaled walkscore field to a scale of 0-100
with arcpy.da.UpdateCursor(walkscore_fishnet, [walkscore_field]) as cursor:
    for row in cursor:
        if max_walkscore > min_walkscore:
            row[0] = ((row[0] - min_walkscore) / (max_walkscore - min_walkscore)) * 100
        else:
            row[0] = 0
        cursor.updateRow(row)

print("Scaled walkscore normalization to 0-100 scale complete.")

Scaled walkscore normalization to 0-100 scale complete.


### Aggregating Walkscore by Neighborhood

In [186]:
# Set environment settings
arcpy.env.workspace = r"C:\Users\rtvpd\OneDrive\Documentos\ArcGIS\Projects\Walkability_Seattle\Walkability_Seattle.gdb"
arcpy.env.overwriteOutput = True  # Allow outputs to be overwritten

# Define the input layers
walkscore_fishnet = r"C:\Users\rtvpd\OneDrive\Documentos\ArcGIS\Projects\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet"
neighborhoods = r"C:\Users\rtvpd\OneDrive\Documentos\ArcGIS\Projects\Walkability_Seattle\Walkability_Seattle.gdb\neighborhoods"
clipped_walkscore_fishnet = "walkscore_fishnet_clipped"
spatial_join_output = "walkscore_neighborhoods_join"
statistics_output = "walkscore_statistics"
output_layer = "walkscore_neighborhoods"

# Step 1: Ensure the spatial reference systems match
walkscore_sr = arcpy.Describe(walkscore_fishnet).spatialReference
neighborhoods_sr = arcpy.Describe(neighborhoods).spatialReference

if walkscore_sr.name != neighborhoods_sr.name:
    raise ValueError("Spatial references do not match between walkscore_fishnet and neighborhoods.")

# Step 2: Clip the walkscore fishnet to the neighborhoods layer
arcpy.analysis.Clip(walkscore_fishnet, neighborhoods, clipped_walkscore_fishnet)
print("Clipped walkscore fishnet to neighborhoods layer.")

# Step 3: Perform a spatial join to associate each fishnet cell with a neighborhood
arcpy.analysis.SpatialJoin(target_features=clipped_walkscore_fishnet,
                           join_features=neighborhoods,
                           out_feature_class=spatial_join_output,
                           join_type="KEEP_COMMON",
                           match_option="INTERSECT")
print("Performed spatial join between neighborhoods and clipped walkscore fishnet.")

# Step 4: Calculate the total walkscore and total area for each neighborhood
arcpy.analysis.Statistics(spatial_join_output, statistics_output,
                          [["walk_score", "SUM"], ["total_area", "SUM"]],
                          "nested")
print("Calculated total walkscore and total area for each neighborhood.")

# Step 5: Add a field for the area-normalized average walkscore
area_normalized_walkscore_field = "area_normalized_walk_score"

# Delete the existing field if it exists
if any(f.name == area_normalized_walkscore_field for f in arcpy.ListFields(neighborhoods)):
    arcpy.management.DeleteField(neighborhoods, area_normalized_walkscore_field)

arcpy.management.AddField(neighborhoods, area_normalized_walkscore_field, "DOUBLE")

# Step 6: Join the statistics table to the neighborhoods layer
arcpy.management.JoinField(in_data=neighborhoods,
                           in_field="nested",
                           join_table=statistics_output,
                           join_field="nested",
                           fields=["SUM_walk_score", "SUM_total_area"])
print("Joined the statistics table to the neighborhoods layer.")

# Step 7: Calculate the area-normalized average walkscore for each neighborhood
with arcpy.da.UpdateCursor(neighborhoods, ["SUM_walk_score", "SUM_total_area", area_normalized_walkscore_field]) as cursor:
    for row in cursor:
        if row[0] is not None and row[1] is not None and row[1] != 0:
            row[2] = row[0] / row[1]  # SUM_walk_score / SUM_total_area
        else:
            row[2] = None
        cursor.updateRow(row)
print("Calculated area-normalized average walkscore for each neighborhood.")

# Step 8: Normalize the area-normalized walkscores to be between 0 and 10
# Step 8.1: Calculate min and max of area-normalized walkscore
scores = [row[0] for row in arcpy.da.SearchCursor(neighborhoods, [area_normalized_walkscore_field]) if row[0] is not None]
min_score = min(scores)
max_score = max(scores)
print(f"Min score: {min_score}, Max score: {max_score}")

# Step 8.2: Add a new field for the normalized walkscore
normalized_walkscore_field = "normalized_walk_score"

# Delete the existing field if it exists
if any(f.name == normalized_walkscore_field for f in arcpy.ListFields(neighborhoods)):
    arcpy.management.DeleteField(neighborhoods, normalized_walkscore_field)

arcpy.management.AddField(neighborhoods, normalized_walkscore_field, "DOUBLE")

# Step 8.3: Normalize the walkscore to be between 0 and 10
with arcpy.da.UpdateCursor(neighborhoods, [area_normalized_walkscore_field, normalized_walkscore_field]) as cursor:
    for row in cursor:
        if row[0] is not None:
            row[1] = 10 * (row[0] - min_score) / (max_score - min_score)
        else:
            row[1] = None
        cursor.updateRow(row)
print("Normalized area-normalized walkscores to be between 0 and 10.")

# Step 9: Export the joined layer to a new feature class
arcpy.conversion.FeatureClassToFeatureClass(in_features=neighborhoods,
                                            out_path=arcpy.env.workspace,
                                            out_name=output_layer)
print("Exported the joined layer to a new feature class with normalized walkscores.")

# Step 10: Clean up temporary fields
fields_to_delete = ["SUM_walk_score", "SUM_total_area", area_normalized_walkscore_field, normalized_walkscore_field]
for field in fields_to_delete:
    if any(f.name == field for f in arcpy.ListFields(neighborhoods)):
        arcpy.management.DeleteField(neighborhoods, field)
        print(f"Deleted temporary field: {field}")

print("Temporary fields cleaned up.")

Clipped walkscore fishnet to neighborhoods layer.
Performed spatial join between neighborhoods and clipped walkscore fishnet.
Calculated total walkscore and total area for each neighborhood.
Joined the statistics table to the neighborhoods layer.
Calculated area-normalized average walkscore for each neighborhood.
Min score: 4.855863430689656e-05, Max score: 0.0015327104969275881
Normalized area-normalized walkscores to be between 0 and 10.
Exported the joined layer to a new feature class with normalized walkscores.
Deleted temporary field: SUM_walk_score
Deleted temporary field: SUM_total_area
Deleted temporary field: area_normalized_walk_score
Deleted temporary field: normalized_walk_score
Temporary fields cleaned up.


#### Smoothing the Walkscore Fishnet

### Adjust Symbology

Now that we have a field that aggregates the walkscore for each component, let's programmatically change the symology of the walkscore_fishnet layer so that we can visually identify which areas have the best walkscore.

In [183]:
# import arcpy
# import os

# aprx_path = r"C:\Users\rtvpd\OneDrive\Documentos\ArcGIS\Projects\Walkability_Seattle\Walkability_Seattle.aprx"
# walkscore_field = 'walk_score'  # The field used to color the fishnet grid
# walkscore_fishnet_layer_path = r"C:\Users\rtvpd\OneDrive\Documentos\ArcGIS\Projects\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet"

# # Check if the project file is writable
# if os.access(aprx_path, os.W_OK):
#     print(f"The project file {aprx_path} is writable.")
# else:
#     print(f"The project file {aprx_path} is not writable. Please check permissions.")

# # Open the ArcGIS Pro project
# aprx = arcpy.mp.ArcGISProject(aprx_path)
# print("Successfully opened the ArcGIS Pro project.")

# # Specify the map name if it's just "Map"
# map_name = "Map"
# m = aprx.listMaps(map_name)[0]

# # Find the walkscore_fishnet layer in the map
# walkscore_layer = None
# for lyr in m.listLayers():
#     if lyr.isFeatureLayer and lyr.name == "walkscore_fishnet":
#         walkscore_layer = lyr
#         break

# # If the walkscore_fishnet layer is not found, add it to the map
# if walkscore_layer is None:
#     print("walkscore_fishnet layer not found in the map. Adding it now.")
#     walkscore_layer = m.addDataFromPath(walkscore_fishnet_layer_path)
#     walkscore_layer.name = "walkscore_fishnet"

# # Apply a graduated color renderer to the walkscore_fishnet layer
# sym = walkscore_layer.symbology
# if hasattr(sym, 'renderer') and sym.renderer.type == 'GraduatedColorsRenderer':
#     sym.updateRenderer('GraduatedColorsRenderer')
#     sym.renderer.classificationField = walkscore_field
#     sym.renderer.breakCount = 5  # You can adjust the number of classes here
#     color_ramp_name = "Greens (Continuous)"
#     color_ramps = aprx.listColorRamps("*")
#     greens_ramp = next((ramp for ramp in color_ramps if ramp.name == color_ramp_name), None)
#     if greens_ramp is not None:
#         sym.renderer.colorRamp = greens_ramp
#         walkscore_layer.symbology = sym
#         print(f"Applied graduated color renderer to {walkscore_layer.name} based on {walkscore_field} using {color_ramp_name}.")
#     else:
#         print(f"Color ramp {color_ramp_name} not found.")
# else:
#     print(f"{walkscore_layer.name} does not support graduated color renderer.")

# # Try to save the project using saveACopy
# try:
#     new_aprx_path = r"C:\Users\rtvpd\OneDrive\Documentos\ArcGIS\Projects\Walkability_Seattle\Walkability_Seattle.aprx"
#     aprx.saveACopy(new_aprx_path)
#     print(f"Project saved with updated layer symbology to {new_aprx_path}.")
# except OSError as e:
#     print(f"Failed to save the project: {e}")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")

In [184]:
# This is where I left off, the fishnet walkscore layer isn't updating with the changed symobology above. Need to troubleshoot.